In [1]:
#WICE

# Women in Continuing Educat

The following cell will install imbalanced-learn so that the project can run Smote to 
over/undersample under/over represented features

In [2]:
!pip install imbalanced-learn

In [3]:
#importing neccesary packages/modules
import warnings
warnings.simplefilter("ignore")
import pandas as pd
pd.options.mode.chained_assignment = None  # default='warn'
import numpy as np
import sklearn as sk
from sklearn.model_selection import train_test_split
from sklearn.model_selection import cross_val_score
from sklearn.tree import DecisionTreeClassifier
from sklearn.metrics import accuracy_score
from sklearn.model_selection import GridSearchCV
from sklearn.naive_bayes import GaussianNB
from sklearn.model_selection import cross_val_predict
from sklearn.metrics import confusion_matrix
from sklearn.metrics import classification_report
from sklearn.preprocessing import StandardScaler
from sklearn.decomposition import PCA
from sklearn.neighbors import KNeighborsClassifier
from imblearn.pipeline import Pipeline
import matplotlib.pyplot as plt
from sklearn.svm import SVC
from sklearn.neural_network import MLPClassifier
from sklearn.ensemble import RandomForestClassifier

%matplotlib inline

In [4]:
#Reads data obtained from Austin PD on crime reports

#skip initial space to not deal with the space in front of every column😎
data = pd.read_csv('newwice.csv', skipinitialspace = True)
pd.set_option('display.max_columns', None)
print("Every Feature: \n", data.columns)
data.head()

Every Feature: 
 Index(['id', 'age', 'education', 'experience', 'employment',
       'out_of_education', 'fufillment', 'industry', 'literacy', 'math',
       'science', 'technology', 'budget', 'hours_per_week', 'program_level',
       'resource1', 'resource2', 'resource3'],
      dtype='object')


,id,age,education,experience,employment,out_of_education,fufillment,industry,literacy,math,science,technology,budget,hours_per_week,program_level,resource1,resource2,resource3
0,1,40,some_college,computer science,student,23,exam,education,1,2,3,3,low,39,3,khan,edx,Princeton Review
1,2,34,less_high_school,secretary,unemployed,10,personal,transportation,2,3,5,4,med,35,3,leetcode,coursera,Magoosh
2,3,52,high_school,accountant,homemaker,3,personal,finance,3,4,4,5,med,20,3,edx,coursera,Princeton Review
3,4,28,bachelors,homemaker,student,8,exam,education,3,1,2,4,med,23,3,khan,coursera,Magoosh
4,5,34,less_high_school,service,unemployed,9,employment,transportation,5,5,3,3,low,24,4,edx,edx,Kaplan


In [12]:
#Perform one hot encoding on all features with categorical data
clean_data = pd.get_dummies(data, columns = ['education', 'experience', 'employment', 'fufillment', 'industry', 'budget'], drop_first = False)
clean_data = clean_data.drop(columns=['id'])
print("Columns After One Hot Encoding")
print(clean_data.columns)



Columns After One Hot Encoding
Index(['age', 'out_of_education', 'literacy', 'math', 'science', 'technology',
       'hours_per_week', 'program_level', 'resource1', 'resource2',
       'resource3', 'education_bachelors', 'education_high_school',
       'education_less_high_school', 'education_some_college',
       'experience_accountant', 'experience_computer science',
       'experience_farming', 'experience_homemaker', 'experience_retail',
       'experience_secretary', 'experience_service', 'experience_teaching',
       'employment_homemaker', 'employment_student', 'employment_unable',
       'employment_unemployed', 'fufillment_employment', 'fufillment_exam',
       'fufillment_personal', 'industry_education', 'industry_finance',
       'industry_manufacturing', 'industry_public_admin',
       'industry_transportation', 'budget_high', 'budget_low', 'budget_med'],
      dtype='object')


In [6]:
#show number of missing/invalid data
print("Invalid Data Counts")
print(data.isna().sum())

print("\nData Shape")
print(data.shape)

Invalid Data Counts
id                  0
age                 0
education           0
experience          0
employment          0
out_of_education    0
fufillment          0
industry            0
literacy            0
math                0
science             0
technology          0
budget              0
hours_per_week      0
program_level       0
resource1           0
resource2           0
resource3           0
dtype: int64

Data Shape
(30, 18)


In [13]:
r1 = clean_data.loc[:,['resource1']]
r2 = clean_data.loc[:,['resource2']]
r3 = clean_data.loc[:,['resource3']]
features_data = clean_data.drop(columns=['resource1', 'resource2', 'resource3'])
features_data

,age,out_of_education,literacy,math,science,technology,hours_per_week,program_level,education_bachelors,education_high_school,education_less_high_school,education_some_college,experience_accountant,experience_computer science,experience_farming,experience_homemaker,experience_retail,experience_secretary,experience_service,experience_teaching,employment_homemaker,employment_student,employment_unable,employment_unemployed,fufillment_employment,fufillment_exam,fufillment_personal,industry_education,industry_finance,industry_manufacturing,industry_public_admin,industry_transportation,budget_high,budget_low,budget_med
0,40,23,1,2,3,3,39,3,0,0,0,1,0,1,0,0,0,0,0,0,0,1,0,0,0,1,0,1,0,0,0,0,0,1,0
1,34,10,2,3,5,4,35,3,0,0,1,0,0,0,0,0,0,1,0,0,0,0,0,1,0,0,1,0,0,0,0,1,0,0,1
2,52,3,3,4,4,5,20,3,0,1,0,0,1,0,0,0,0,0,0,0,1,0,0,0,0,0,1,0,1,0,0,0,0,0,1
3,28,8,3,1,2,4,23,3,1,0,0,0,0,0,0,1,0,0,0,0,0,1,0,0,0,1,0,1,0,0,0,0,0,0,1
4,34,9,5,5,3,3,24,4,0,0,1,0,0,0,0,0,0,0,1,0,0,0,0,1,1,0,0,0,0,0,0,1,0,1,0
5,33,7,4,2,1,2,25,4,0,0,0,1,0,0,0,0,1,0,0,0,0,0,0,1,1,0,0,0,1,0,0,0,1,0,0
6,60,3,1,3,2,1,10,5,0,0,0,1,0,0,0,0,1,0,0,0,1,0,0,0,1,0,0,0,0,0,0,1,1,0,0
7,78,4,1,3,2,2,40,2,0,1,0,0,0,1,0,0,0,0,0,0,1,0,0,0,1,0,0,0,1,0,0,0,0,1,0
8,62,45,2,3,5,2,45,2,0,1,0,0,0,0,1,0,0,0,0,0,0,0,0,1,0,1,0,0,0,1,0,0,0,1,0
9,45,21,5,2,5,3,38,2,0,0,1,0,0,0,0,0,1,0,0,0,1,0,0,0,1,0,0,0,1,0,0,0,0,0,1


In [9]:
corr_matrix = clean_data.corr()
corr_list = []
print("Correlation of Features with a value greater than 0.6 or less than -0.6")
for i in range(len(corr_matrix.columns)):
    for j in range(i):
        if abs(corr_matrix.iloc[i, j]) > 0.6: #print out high correlation
            corr_list.append([corr_matrix.columns[i], corr_matrix.columns[j], corr_matrix.iloc[i, j]])

corr_frame = pd.DataFrame(corr_list, columns = ['Feature 1', 'Feature 2', 'Correlation'])
corr_frame = corr_frame.style.set_precision(4)
display(corr_frame)

Correlation of Features with a value greater than 0.6 or less than -0.6


,Feature 1,Feature 2,Correlation
0,literacy,age,-0.6475
1,program_level,hours_per_week,-0.6658
2,experience_farming,out_of_education,0.8281
3,employment_homemaker,age,0.7011
4,fufillment_employment,technology,-0.6163
5,fufillment_exam,fufillment_employment,-0.6667
6,fufillment_personal,technology,0.7009
7,industry_education,employment_student,1.0000
8,industry_manufacturing,out_of_education,0.8281
9,industry_manufacturing,experience_farming,1.0000


In [31]:

scaler = StandardScaler()
neighbor = KNeighborsClassifier(n_neighbors = 3)

pip = Pipeline(steps = [('scaler',scaler),('knn',neighbor)])
param_grid = {
    'knn__n_neighbors': list(range(1, 3))
}
grid = GridSearchCV(pip, param_grid, cv = 5, scoring = 'accuracy')
knn_pred = cross_val_predict(grid, features_data, r1, cv = 5)
print("Accuracy: \n", accuracy_score(r1, knn_pred)*100)
print("Confusion Matrix: \n", confusion_matrix(r1, knn_pred))
print()
print("Classification Report: \n", classification_report(r1, knn_pred))

print('Optimized KNN using Smote')
pip = Pipeline(steps = [('smote', smote),('scaler',scaler),('pca',pca),('knn',neighbor)])
grid = GridSearchCV(pip, param_grid, cv = 5,  scoring = 'accuracy')
knn_pred = cross_val_predict(grid, features_data, r2, cv = 5)
print("Accuracy: \n", accuracy_score(r2, knn_pred)*100)
print("Confusion Matrix: \n", confusion_matrix(labels_data, knn_pred))
print()
print("Classification Report: \n", classification_report(r2, knn_pred))

print("Optimized KNN using Smote and Recall")
grid = GridSearchCV(pip, param_grid, cv = 5,  scoring = 'recall')
knn_pred = cross_val_predict(grid, features_data, r3, cv = 5)
print("Accuracy: \n", accuracy_score(r3, knn_pred)*100r3
print("Confusion Matrix: \n", confusion_matrix(labels_data, knn_pred))
print()
print("Classification Report: \n", classification_report(r3, knn_pred))

Accuracy: 
 53.333333333333336
Confusion Matrix: 
 [[4 3 2]
 [4 5 1]
 [2 2 7]]



NameError: name 'labels_data' is not defined

In [19]:
import imblearn
from imblearn.over_sampling import SMOTE

x_train, x_test, y_train, y_test = train_test_split(features_data, r1, train_size = 0.8)
print("\nShapes of Test/Train Sets")
print("x_train: ", x_train.shape)
print("x_test: ", x_test.shape)
print("y_train: ", y_train.shape)
print("y_test: ", y_test.shape)

tree_classifier = DecisionTreeClassifier(criterion = 'entropy')
x_train, y_train = SMOTE().fit_resample(x_train, y_train)
print("after smote distribution is now ", y_train['resource1'].value_counts())
tree_classifier.fit(x_train, y_train)

x_train, x_test, y_train, y_test = train_test_split(features_data, r2, train_size = 0.8)
print("\nShapes of Test/Train Sets")
print("x_train: ", x_train.shape)
print("x_test: ", x_test.shape)
print("y_train: ", y_train.shape)
print("y_test: ", y_test.shape)
print("after smote distribution is now ", y_train['resource2'].value_counts())
tree_classifier.fit(x_train, y_train)
prediction = tree_classifier.predict(x_test)


x_train, x_test, y_train, y_test = train_test_split(features_data, r3, train_size = 0.8)
print("\nShapes of Test/Train Sets")
print("x_train: ", x_train.shape)
print("x_test: ", x_test.shape)
print("y_train: ", y_train.shape)
print("y_test: ", y_test.shape)
print("after smote distribution is now ", y_train['resource3'].value_counts())
tree_classifier.fit(x_train, y_train)
prediction = tree_classifier.predict(x_test)


Shapes of Test/Train Sets
x_train:  (24, 35)
x_test:  (6, 35)
y_train:  (24, 1)
y_test:  (6, 1)
after smote distribution is now  leetcode    9
edx         9
khan        9
Name: resource1, dtype: int64

Shapes of Test/Train Sets
x_train:  (24, 35)
x_test:  (6, 35)
y_train:  (24, 1)
y_test:  (6, 1)
after smote distribution is now  coursera    11
edx         10
openware     3
Name: resource2, dtype: int64

Shapes of Test/Train Sets
x_train:  (24, 35)
x_test:  (6, 35)
y_train:  (24, 1)
y_test:  (6, 1)
after smote distribution is now  Kaplan              11
Magoosh              6
Princeton Review     4
Manhattan Prep       3
Name: resource3, dtype: int64


In [24]:
bayes_classifier = GaussianNB()
pip = Pipeline(steps = [('bayes',bayes_classifier)])
print("r1")
naive_cross = cross_val_score(pip, features_data, r1, cv = 10)
print("Accuracy: ", naive_cross.mean() * 100)
naive_pred = cross_val_predict(pip, features_data, r1, cv = 10)
print("Confusion Matrix: \n", confusion_matrix(r1, naive_pred))
print()
print("Classification Report: \n", classification_report(r1, naive_pred))

print("r2")
naive_cross = cross_val_score(pip, features_data, r2, cv = 10)
print("Accuracy: ", naive_cross.mean() * 100)
naive_pred = cross_val_predict(pip, features_data, r3, cv = 10)
print("Confusion Matrix: \n", confusion_matrix(r1, naive_pred))
print()
print("Classification Report: \n", classification_report(r1, naive_pred))


print("r3")
naive_cross = cross_val_score(pip, features_data, r3, cv = 10)
print("Accuracy: ", naive_cross.mean() * 100)
naive_pred = cross_val_predict(pip, features_data, r3, cv = 10)
print("Confusion Matrix: \n", confusion_matrix(r1, naive_pred))
print()
print("Classification Report: \n", classification_report(r1, naive_pred))


r1
Accuracy:  56.666666666666664
Confusion Matrix: 
 [[4 1 4]
 [4 5 1]
 [2 1 8]]

Classification Report: 
               precision    recall  f1-score   support

         edx       0.40      0.44      0.42         9
        khan       0.71      0.50      0.59        10
    leetcode       0.62      0.73      0.67        11

    accuracy                           0.57        30
   macro avg       0.58      0.56      0.56        30
weighted avg       0.58      0.57      0.57        30

r2
Accuracy:  53.33333333333334
Confusion Matrix: 
 [[0 0 0 0 0 0 0]
 [0 0 0 0 0 0 0]
 [0 0 0 0 0 0 0]
 [0 0 0 0 0 0 0]
 [3 2 2 2 0 0 0]
 [2 6 0 2 0 0 0]
 [4 6 1 0 0 0 0]]

Classification Report: 
                   precision    recall  f1-score   support

          Kaplan       0.00      0.00      0.00       0.0
         Magoosh       0.00      0.00      0.00       0.0
  Manhattan Prep       0.00      0.00      0.00       0.0
Princeton Review       0.00      0.00      0.00       0.0
             edx       